In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

In [3]:
schedule = sc.textFile("../Data/FoodTruck_Schedule/Mobile_Food_Schedule.csv")\
             .map(lambda x : x.split(","))
day_string = sc.textFile("../Data/FoodTruck_Schedule/Day_Num_String.csv")\
               .map(lambda x : x.split(","))

In [5]:
day_string.first()

['0', 'Sunday', '', '', '', '', '']

## Create a pair RDD to make sure the same "day" goes to the same partition

In [10]:
schedulee_pair = schedule.map(lambda x : (x[0], x[1:])).partitionBy(7).cache()
day_pair = day_string.map(lambda x : (x[0], x[1:])).partitionBy(7).cache()

In [11]:
schedulee_pair.keys().glom().collect()

[['1',
  '1',
  '2',
  '1',
  '1',
  '2',
  '1',
  '2',
  '1',
  '2',
  '2',
  '2',
  '1',
  '1',
  '2',
  '1',
  '2',
  '1',
  '1',
  '1',
  '1',
  '1',
  '1',
  '2',
  '1',
  '2',
  '1',
  '2',
  '2',
  '2',
  '1',
  '2',
  '2',
  '2',
  '1',
  '1',
  '1',
  '1',
  '1',
  '2',
  '2',
  '2',
  '1',
  '2',
  '2',
  '1',
  '1',
  '2',
  '2',
  '1',
  '2',
  '1',
  '1',
  '1',
  '2',
  '2',
  '2',
  '2',
  '1',
  '2',
  '1',
  '2',
  '1',
  '2',
  '1',
  '2',
  '1',
  '1',
  '1',
  '1',
  '2',
  '2',
  '1',
  '2',
  '2',
  '1',
  '1',
  '1',
  '2',
  '1',
  '2',
  '1',
  '2',
  '1',
  '2',
  '2',
  '2',
  '2',
  '1',
  '2',
  '2',
  '1',
  '1',
  '1',
  '2',
  '2',
  '1',
  '2',
  '1',
  '1',
  '2',
  '1',
  '1',
  '2',
  '2',
  '2',
  '1',
  '1',
  '2',
  '2',
  '1',
  '1',
  '1',
  '1',
  '1',
  '1',
  '2',
  '2',
  '1',
  '2',
  '2',
  '1',
  '2',
  '2',
  '2',
  '1',
  '1',
  '1',
  '2',
  '2',
  '2',
  '2',
  '1',
  '2',
  '2',
  '1',
  '2',
  '2',
  '1',
  '1',
  '2',
  '1',
  '2',

In [14]:
day_pair.keys().glom().collect()

[['1', '2'], ['4'], ['5', '6'], ['0', '3']]

In [13]:
day_pair = day_pair.coalesce(4)
schedulee_pair = schedulee_pair.coalesce(4)

## Return the Mobile_Food_Schedule info with "day string" instead of numeric values

In [17]:
scheulde_day_string = day_pair.join(schedulee_pair).map(lambda x : (x[1][0][0], x[1][1]))

In [18]:
scheulde_day_string.first()

('Thursday',
 ['6AM',
  '11PM',
  '19MFF-00127',
  '1 MONTGOMERY ST',
  '1382718',
  '',
  '06:00',
  '23:00',
  '10624001',
  '12/10/2019 02:32:33 PM',
  '12/10/2019 02:34:15 PM',
  '292',
  '2',
  'N',
  '37.789121900046300',
  '-122.40207154934200'])

## Return the number of total scheuldes with the count of "monday" schedules 

In [19]:
monday_count = sc.accumulator(0)
total_count = sc.accumulator(0)

In [21]:
total_count.value

0

In [22]:
def count(x):
    if(x[0] == 'Monday'):
        monday_count.add(1)
    total_count.add(1)

In [23]:
scheulde_day_string.foreach(count)

In [24]:
monday_count.value

584

In [25]:
total_count.value

3137